# Оптимизация выполнения кода, векторизация, Numba

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. "Оптимизация выполнения кода, векторизация, Numba"
* IPython Cookbook, Second Edition (2018), глава 4
* https://ipython-books.github.io/43-profiling-your-code-line-by-line-with-line_profiler/
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html
* https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

## Задачи для совместного разбора

In [5]:
!pip install git+https://github.com/pyutils/line_profiler.git
!pip install line_profiler
!pip install --user numpy==1.20

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/pyutils/line_profiler.git to /tmp/pip-req-build-cv6p_ub7
  Running command git clone -q https://github.com/pyutils/line_profiler.git /tmp/pip-req-build-cv6p_ub7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for line-profiler: filename=line_profiler-4.0.2-cp38-cp38-linux_x86_64.whl size=460004 sha256=45b03b220377541eef3c8acb3225bb13881b42399d8049544dab40ce58d553c3
  Stored in directory: /tmp/pip-ephem-wheel-cache-0jz0b9nb/wheels/d3/20/57/fccf8f1a95000db18d57f1c7b8d0b78d33020f82f50f7b26f6
Successfully built line-profiler
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import numpy as np
import numba
from numba import njit

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

2. Напишите функцию, которая возвращает сумму всех чисел от 0 до x-1. Примените функцию к каждому элементу массива.

3. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [7]:
%load_ext line_profiler

In [8]:
import pandas as pd
import string

N = 2_000_000
df = pd.DataFrame(np.random.randn(N, 4), columns=[f"col{i}" for i in range(4)])
df["key"] = np.random.choice(list(string.ascii_letters.lower()), N, replace=True)
df.head(2)

,col0,col1,col2,col3,key
0,0.839492,0.807467,1.481977,-1.071109,h
1,-0.144447,0.068865,1.064956,-0.701123,w


## Лабораторная работа 8

In [9]:
import numpy as np
import pandas as pd
from numba import jit, njit, guvectorize
import numba
from typing import Union
import re

%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

## Измерение времени выполнения кода

Назовем полным описанием рецепта строку, полученную путем конкатенации названия и описания рецепта через пробел. Удалите строки для рецептов, которые были добавлены не в 2010 году.

Реализуйте несколько вариантов функции подсчета средней длины полного описания рецепта для рецептов, добавленных в 2010 году.

In [26]:
recipes = pd.read_csv('recipes_sample (1).csv', parse_dates = ['submitted'])
reviews = pd.read_csv('reviews_sample (1).csv')

In [27]:
recipes

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16.0,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,NaN,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,NaN,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,227978,2012-08-29,NaN,this is a delicious soup that i originally fou...,NaN


In [28]:
reviews

,Unnamed: 0,user_id,recipe_id,date,rating,review
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
...,...,...,...,...,...,...
126691,1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...
126692,158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...
126693,1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou..."
126694,453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...


In [56]:
recipes['submitted'] = pd.to_datetime(recipes.submitted)
new_recipes = recipes[recipes.submitted.dt.year == 2010]

In [57]:
new_recipes 

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
52,just peachy cobbler,437637,70,1085867,2010-09-17,10.0,all i can say is yummmmmm . . . a simple to ma...,10.0
68,the heat spicy party mix,437219,95,1682162,2010-09-13,NaN,a spicy chex mix that will really warm your gu...,11.0
81,iowa state fair sweet dough caramel cinnamon ...,435816,80,17803,2010-08-24,29.0,this was the winning entry at the 2010 iowa st...,NaN
104,1 minute blueberries cream,428566,2,1375473,2010-06-04,4.0,i was craving blueberry tonight but wanted non...,NaN
146,2 2 2 diet mocha,416599,5,789314,2010-03-15,5.0,"while trying to come up with a satisfying ""sna...",7.0
...,...,...,...,...,...,...,...,...
29897,zoe s chicken tarragon,441211,40,76559,2010-11-04,12.0,from a good housekeeping at my hair salon. ha...,9.0
29907,zucchini and noodle slice,412518,60,423555,2010-02-10,21.0,"a yummy, tasty slice packed with vegies and ri...",13.0
29915,zucchini bread bread machine,409757,220,539686,2010-01-22,7.0,"originally from a packet of red star yeast, th...",NaN
29926,zucchini chip cupcakes,406686,35,628076,2010-01-04,9.0,this is a great tasting recipe to use up zucch...,14.0


1\.1 С использованием метода `DataFrame.iterrows` таблицы:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - нахождение полного описания рецепта осуществляется внутри цикла по `iterrows` для каждой строки по отдельности.

In [88]:
def get_mean_len_A(df):
    s = k = 0
    for i in df.iterrows():
        s += len(i[1][0]) + len(i[1][6]) + 1
        k += 1
    return s/k


In [89]:
get_mean_len_A(df)

265.501300390117

1\.2. С использованием метода `DataFrame.apply` таблицы:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - вызываете метод apply у таблицы, в качестве аргумента передаете функцию, которая возвращает полное описание для каждой строки;
    
    - считаете среднюю длину описаний, вызвав соответствующий метод серии.

In [122]:
def get_mean_len_B(Data):
    sum_ = (df.apply(lambda x: x["name"] + x.description, axis=1).str.len())
    return  sum(sum_/len(df))

In [123]:
get_mean_len_B(new_recipes)

264.5013003901173

In [124]:
def get_mean_len_B2(Data):
    return df.apply(lambda x: len(x['name']) + len(x.description), axis=1).mean()

In [125]:
get_mean_len_B2(df)

264.501300390117

1\.3. С использованием векторизованных методов серий `pd.Series`:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - при помощи векторизированных операций получаете столбец с полным описанием;
    
    - при помощи векторизированных операций получаете длины полного описания;
    
    - при помощи метода серий получаете среднюю длину полных описаний. 

In [126]:
def get_mean_len_C(Data):
    return (df['name'] + df.description).str.len().mean()

In [119]:
get_mean_len_C(new_recipes)

265.501300390117

1.4 Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций при помощи магических команд `time` и `timeit`.

In [140]:
%time get_mean_len_A(new_recipes)

CPU times: user 78.1 ms, sys: 953 µs, total: 79.1 ms
Wall time: 82.1 ms


265.501300390117

In [141]:
%time get_mean_len_B(new_recipes)

CPU times: user 38.6 ms, sys: 0 ns, total: 38.6 ms
Wall time: 46.2 ms


264.5013003901173

In [142]:
%time get_mean_len_C(new_recipes)

CPU times: user 4.6 ms, sys: 0 ns, total: 4.6 ms
Wall time: 4.52 ms


264.501300390117

## Анализ пошагового выполнения кода 

Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. 

In [132]:
import re


def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=["review"]).iterrows():
        recipe_id, review = row["recipe_id"], row["review"]
        words = re.sub("[^A-Za-z\s]", "", review).split(" ")
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)

    word_reviews_count = {}
    for _, row in df.dropna(subset=["review"]).iterrows():
        review = row["review"]
        words = re.sub("[^A-Za-z\s]", "", review).split(" ")
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

2.1 Найдите узкие места в коде, проанализировав код функции по шагам, используя профайлер. Сохраните результаты работы профайлера в отдельную текстовую ячейку. Выпишите (словами), что в имеющемся коде реализовано неоптимально. 

2.2  Оптимизируйте функцию и добейтесь значительного (как минимум, в 5 раз) прироста в скорости выполнения. Для демонстрации результата измерьте скорость выполнения оригинальной функции и функции, написанной вами.

In [136]:
def get_word_reviews_count_opt(df):
    word_reviews_count = {}
    for row in df['review'].dropna():
        #print(row)
        words = re.sub("[^A-Za-z\s]", "", row).split(" ")
        for word in words:
            word_reviews_count[word] = word_reviews_count.get(word,0)+1
    return word_reviews_count

In [139]:
%time get_word_reviews_count(new_recipes)

CPU times: user 22.3 s, sys: 144 ms, total: 22.4 s
Wall time: 23.3 s


{'Last': 97,
 'week': 1509,
 'whole': 5782,
 'sides': 458,
 'of': 109239,
 'frozen': 2943,
 'salmon': 1032,
 'fillet': 92,
 'was': 89617,
 'on': 36121,
 'sale': 262,
 'in': 62796,
 'my': 44644,
 'local': 572,
 'supermarket': 95,
 'so': 46848,
 'I': 288243,
 'bought': 1514,
 'tons': 163,
 'okay': 600,
 'only': 14209,
 '': 330966,
 'but': 43035,
 'total': 512,
 'weight': 216,
 'over': 9801,
 'pounds': 291,
 'This': 39555,
 'recipe': 71751,
 'is': 56519,
 'perfect': 7807,
 'for': 123162,
 'even': 8074,
 'though': 4942,
 'it': 133219,
 'calls': 525,
 'steaks': 511,
 'cut': 6848,
 'up': 16178,
 'the': 266530,
 'into': 7085,
 'individual': 314,
 'portions': 214,
 'and': 219215,
 'followed': 4880,
 'instructions': 1007,
 'exactly': 4625,
 'Im': 8330,
 'one': 17615,
 'those': 2419,
 'food': 3498,
 'combining': 78,
 'diets': 45,
 'left': 5064,
 'out': 25697,
 'white': 3690,
 'wine': 1760,
 'added': 22099,
 'just': 25271,
 'a': 166467,
 'dash': 547,
 'vinegar': 1946,
 'instead': 11644,
 'little'

In [ ]:
%time get_word_reviews_count_opt(new_recipes)

2.43 s ± 73 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [144]:
get_word_reviews_count(new_recipes)

{'Last': 97,
 'week': 1509,
 'whole': 5782,
 'sides': 458,
 'of': 109239,
 'frozen': 2943,
 'salmon': 1032,
 'fillet': 92,
 'was': 89617,
 'on': 36121,
 'sale': 262,
 'in': 62796,
 'my': 44644,
 'local': 572,
 'supermarket': 95,
 'so': 46848,
 'I': 288243,
 'bought': 1514,
 'tons': 163,
 'okay': 600,
 'only': 14209,
 '': 330966,
 'but': 43035,
 'total': 512,
 'weight': 216,
 'over': 9801,
 'pounds': 291,
 'This': 39555,
 'recipe': 71751,
 'is': 56519,
 'perfect': 7807,
 'for': 123162,
 'even': 8074,
 'though': 4942,
 'it': 133219,
 'calls': 525,
 'steaks': 511,
 'cut': 6848,
 'up': 16178,
 'the': 266530,
 'into': 7085,
 'individual': 314,
 'portions': 214,
 'and': 219215,
 'followed': 4880,
 'instructions': 1007,
 'exactly': 4625,
 'Im': 8330,
 'one': 17615,
 'those': 2419,
 'food': 3498,
 'combining': 78,
 'diets': 45,
 'left': 5064,
 'out': 25697,
 'white': 3690,
 'wine': 1760,
 'added': 22099,
 'just': 25271,
 'a': 166467,
 'dash': 547,
 'vinegar': 1946,
 'instead': 11644,
 'little'

In [ ]:
get_word_reviews_count_opt(new_recipes)

{'Last': 97,
 'week': 1509,
 'whole': 5782,
 'sides': 458,
 'of': 109239,
 'frozen': 2943,
 'salmon': 1032,
 'fillet': 92,
 'was': 89617,
 'on': 36121,
 'sale': 262,
 'in': 62796,
 'my': 44644,
 'local': 572,
 'supermarket': 95,
 'so': 46848,
 'I': 288243,
 'bought': 1514,
 'tons': 163,
 'okay': 600,
 'only': 14209,
 '': 330966,
 'but': 43035,
 'total': 512,
 'weight': 216,
 'over': 9801,
 'pounds': 291,
 'This': 39555,
 'recipe': 71751,
 'is': 56519,
 'perfect': 7807,
 'for': 123162,
 'even': 8074,
 'though': 4942,
 'it': 133219,
 'calls': 525,
 'steaks': 511,
 'cut': 6848,
 'up': 16178,
 'the': 266530,
 'into': 7085,
 'individual': 314,
 'portions': 214,
 'and': 219215,
 'followed': 4880,
 'instructions': 1007,
 'exactly': 4625,
 'Im': 8330,
 'one': 17615,
 'those': 2419,
 'food': 3498,
 'combining': 78,
 'diets': 45,
 'left': 5064,
 'out': 25697,
 'white': 3690,
 'wine': 1760,
 'added': 22099,
 'just': 25271,
 'a': 166467,
 'dash': 547,
 'vinegar': 1946,
 'instead': 11644,
 'little'

## Numba

In [145]:
import json

В файле `rating_predictions.json` хранятся данные о рейтингах рецептов и прогнозных значениях рейтингов для этого рецепта, полученных при помощи модели машинного обучения. 

Напишите несколько версий функции (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от прогнозного значения рейтинга для данного рецепта. 


Замечание 1: в формуле MAPE под $A_t$ понимается рейтинг из отзыва $t$, под $F_t$ - прогнозное значения рейтинга отзыва $t$.

Замечание 2: в результате работы функций должно получиться одно число - MAPE для всего набора данных.

In [150]:
f =json.load(open('rating_predictions.json'))
f

[{'rating': 5.0, 'prediction': 4.944444444444445},
 {'rating': 5.0, 'prediction': 4.4375},
 {'rating': 5.0, 'prediction': 4.7272727272727275},
 {'rating': 5.0, 'prediction': 4.3545454545454545},
 {'rating': 5.0, 'prediction': 4.888888888888889},
 {'rating': 5.0, 'prediction': 4.4855072463768115},
 {'rating': 3.0, 'prediction': 4.666666666666667},
 {'rating': 5.0, 'prediction': 4.594936708860759},
 {'rating': 5.0, 'prediction': 5.0},
 {'rating': 5.0, 'prediction': 4.4},
 {'rating': 4.0, 'prediction': 4.0},
 {'rating': 5.0, 'prediction': 5.0},
 {'rating': 5.0, 'prediction': 4.666666666666667},
 {'rating': 5.0, 'prediction': 3.6666666666666665},
 {'rating': 5.0, 'prediction': 4.666666666666667},
 {'rating': 4.0, 'prediction': 4.403381642512077},
 {'rating': 2.0, 'prediction': 3.5},
 {'rating': 4.0, 'prediction': 4.75},
 {'rating': 5.0, 'prediction': 5.0},
 {'rating': 3.0, 'prediction': 4.0},
 {'rating': 5.0, 'prediction': 5.0},
 {'rating': 5.0, 'prediction': 4.090909090909091},
 {'rating'

3\.1 Создайте два списка `A_list` и `F_list` на основе файла `rating_predictions.json`. Напишите функцию `mape_lists` без использования векторизованных операций и методов массивов `numpy` и без использования `numba` (проитерируйтесь по спискам и вычислите суммарное значение MAPE для всех элементов, а потом усредните результат).

Измерьте время выполнения данной функции на входных данных `A_list` и `F_list`. Временем, затрачиваемым на создание списков, можно пренебречь.
    

In [169]:
A_list = []
F_list = []
for i in f:
  A_list.append(i['rating'])
  F_list.append(i['prediction'])

In [170]:
def mape_lists(A_list, F_list):
    s = 0
    for i in range(len(A_list)):
        s += abs((A_list[i] - F_list[i])/A_list[i])
    return 100/len(A_list) * s

In [171]:
mape_lists(A_list, F_list)

13.325265503992638

In [163]:
%time mape_lists(A_list, F_list)

CPU times: user 29.8 ms, sys: 0 ns, total: 29.8 ms
Wall time: 30.4 ms


13.325265503992638

3\.2. Создайте массивы `numpy` `A_array` и `F_array` на основе списков `A_list` и `F_list`. Напишите функцию `mape_numpy` с использованием векторизованных операций и методов массивов `numpy`.

Измерьте время выполнения данной функции на входных данных `A_array` и `F_array`. Временем, затрачиваемым на создание массивов, можно пренебречь.

In [176]:
A_array = np.array(A_list)
F_array = np.array(F_list)

In [174]:
A_array

array([5., 5., 5., ..., 5., 5., 5.])

In [175]:
F_array 

array([4.94444444, 4.4375    , 4.72727273, ..., 5.        , 4.14285714,
       5.        ])

In [177]:
def mape_numpy(A_array, F_array):
    return 100/len(A_array) * (abs((A_array - F_array) / A_array)).sum()

In [178]:
%timeit mape_numpy(A_array, F_array)

533 µs ± 13.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


3\.3. Создайте объекты `numba.typed.List` `A_typed` и `F_typed` на основе списков `A_list` и `F_list`. Напишите функцию `mape_numba` без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`. 

Измерьте время выполнения данной функции на входных данных `A_typed` и `F_typed`. Временем, затрачиваемым на создание объектов `numba.typed.List`, можно пренебречь.

Измерьте время выполнения данной функции на входных данных `A_array` и `F_array`.

In [180]:
A_typed = numba.typed.List(A_list)
F_typed = numba.typed.List(F_list)

In [181]:
@njit
def mape_numba(A_typed, F_typed):
    s = 0
    for i in range(len(A_typed)):
        s += abs((A_typed[i]-F_typed[i])/A_typed[i])
    return 100/len(A_typed) * s

In [182]:
%time mape_numba(A_typed, F_typed)

CPU times: user 327 ms, sys: 1.88 ms, total: 329 ms
Wall time: 332 ms


13.325265503992638

In [183]:
%time mape_numba(A_array, F_array)

CPU times: user 131 ms, sys: 0 ns, total: 131 ms
Wall time: 136 ms


13.325265503992638

## Векторизация

Сайт-агрегатор устроил акцию: он дарит купоны на посещение ресторана тем пользователям, оставившим отзывы, идентификатор которых является _красивым числом_. Натуральное число называется _красивым_, если первая цифра числа совпадает с последней цифрой числа. 



4\.1 Напишите функцию `is_pretty`, которая для каждого идентификатора пользователя из файла определяет, получит ли он подарок. Запрещается преобразовывать идентификатор пользователя к строке. Подтвердите корректность реализации, продемонстрировав примеры.

In [184]:
ids = reviews["recipe_id"].values

In [186]:
def is_pretty(n):
    F = n
    while F//10 != 0:
        F //= 10
    return n%10 == F

In [190]:
is_pretty(69)

False

In [189]:
is_pretty(121)

True

4\.2 Посчитайте с помощью функции `is_pretty` количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.

In [191]:
res4_2 = list(map(is_pretty, ids))
sum(res4_2)

11955

In [194]:
%time res4_2 =list(map(is_pretty, ids))


CPU times: user 432 ms, sys: 0 ns, total: 432 ms
Wall time: 469 ms


4\.3. При помощи `numpy` создайте векторизованную версию функции `is_pretty`. Посчитайте с помощью этой векторизованной функции количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.


In [ ]:
def is_pretty4_3(arr):
    last = arr % 10
    first = ids // 10**np.vectorize(int)(np.log10(ids))
    return last==first

In [ ]:
is_pretty4_3(ids).sum()

11955

In [ ]:
%%timeit
is_pretty4_3(ids).sum()

16 ms ± 144 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


4\.4. При помощи `numba` создайте векторизованную версию функции `is_pretty`. Посчитайте с помощью этой векторизованной функции количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.
